<img src='IMG/TUD_logo.png' align='right' width='15%'></img>

[Zurück zur Übersicht BHYWI04 - Grundlagen der Wasserbewirtschaftung](BHYWI04_00_index.ipynb)   

# 1D Advektion-Dispersion
## Analytische Lösung
Das Jupyter Notebook erlaubt es, die 1D-Transportgleichung für Advektion und Dispersion zu lösen. Dabei können die transportrelevanten Paramter $\alpha$ (Dispersivität) und $n$ (Porosität) modifiziert werden.

Jupyter Notebook von Thomas Reimann & Max Rudolph, basierend auf einem XLS-Worksheet von Prof. Rudolf Liedl. _Optimal geeignet als Voila Dashboard_
<hr>

In [137]:
# %matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from scipy import special
import numpy as np
from ipywidgets import *
from ipynb.fs.full.BHYWI04_functions import create_multipleChoice_widget
from ipynb.fs.full.BHYWI04_functions import create_multipleChoice_widget_PDF

## Anpassung der analytischen Lösung
Sie können das nachfolgende Diagramm mit den Eingabefeldern und Schiebereglern benutzen. Die Gleichungen berechnen die Konzentration in einer bestimmten Entfernung von der Zugabestelle (Länge) als Funktion der Zeit (= 'Durchbruchskurve')

Sie können ebenfalls verschiedene Berechnungen (Advektion sowie Advektion-Dispersion) und gemessene Werte plotten.

In [142]:
#FUNCTIONS FOR COMPUTATION; ADS = ADVECTION, DISPERSION AND SORPTION - EVENTUALLY SET RETARDATION TO 1 FOR NO SORPTION

def IC(PE,r_time):
    IC1 = np.sqrt(0.25*PE/r_time)*(1-r_time)
    if (IC1>0):
        IC2 = 1-(1-special.erfc(abs(IC1)))
    else:
        IC2 = 1+(1-special.erfc(abs(IC1)))
    IC3 = np.sqrt(0.25*PE/r_time)*(1+r_time)
    if (IC3>0):
        IC4 = 1-(1-special.erfc(abs(IC3)))
    else:
        IC4 = 1+(1-special.erfc(abs(IC3)))
    if IC4 == 0:
        IC5 = IC2
    else:
        IC5 = IC2+np.exp(PE)*IC4
    IC  = 1-0.5*IC5
    return IC

def BC(PE,r_time, r_dur):
    # BCx positive pulse
    BC1 = np.sqrt(0.25*PE/r_time)*(1-r_time)
    if (BC1>0):
        BC2 = 1-(1-special.erfc(abs(BC1)))
    else:
        BC2 = 1+(1-special.erfc(abs(BC1)))
    BC3 = np.sqrt(0.25*PE/r_time)*(1+r_time)
    BC4 = special.erfc(BC3)
    if BC4 == 0:
        BC5 = BC2
    else:
        BC5 = BC2 + np.exp(PE) * BC4
    
    # BCCx negative pulse
    if r_time > r_dur:
        BCC1 = np.sqrt(0.25 * PE / (r_time - r_dur)) * (1 - (r_time - r_dur))
        if BCC1 > 0:
            BCC2 = 1 - (1 - special.erfc(abs(BCC1)))
        else:
            BCC2 = 1 + (1 - special.erfc(abs(BCC1)))  
        BCC3 = np.sqrt(0.25 * PE / (r_time - r_dur)) * (1 + (r_time - r_dur))
        BCC4 = special.erfc(BCC3)
        if BCC4 == 0:
            BCC5 = BCC2
        else:
            BCC5 = BCC2 + np.exp(PE) * BCC4      
    if r_time <= r_dur:
        BC = 0.5 * BC5
    else:
        BC = 0.5 * (BC5 - BCC5)
    return BC

def transport(l,t1,c0,m,Q,n,a,plot_A, plot_AD, plot_DATA):
    # l : länge der säule, float
    # t1 : versuchsende, float
    # ci : initiale konzentration, float
    # c0 : eingabekonzentration, float
    # cp : max. conc plot
    # m : eingabemasse, float
    # Q : durchfluss, float
    # n : porosität, float
    # a : dispersivität (a > 0), float
    
    # Data for plotting
    t0 = 1       #Startzeit
    dt = 2      #Zeitdiskretisierung
    r  = 2    #Radius der Säule
    ci = 0
    cp = 2 * c0
    
    #Berechnung Zwischenergebnisse
    A =     np.pi*r**2
    q =     Q/A
    v =     q/n
    D =     a*v
    PE =    l/a
    dur =   m/Q/(c0-ci)
    tPV =   l/v
    r_dur = dur/tPV
    r_dt =  dt/tPV

    #Festlegung Zeitbereich
    t = np.arange(t0, t1, dt)

    #Berechnung Konzentration - Klammerterme
    #Set fraction of distance
    r_time = []
    time   = []
    conc   = []
    conca  = []
    
    #compute concentration  
    for t in range(t0, t1, dt):      
        r_time = t/tPV
        # ADVECTION-DISPERSION
        c = ci*IC(PE,r_time)+c0*BC(PE,r_time, r_dur)
        conc.append(c)
        # ADVECTION ONLY
        if r_time < 1:
            ca = 0
        elif r_time > 1+r_dur:
            ca = 0
        else:
            ca = c0
        conca.append(ca)
        time.append(t)
        
    # measurements
    t_obs = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
    c_obs = [1e-3, 5e-2, 8.5e-2, 9.7e-2, 9.9e-2, 9e-2, 5e-2, 1.5e-2, 2e-3, 5e-4]
   
    #PLOT FIGURE
    fig = plt.figure(figsize=(9,6))
    ax = fig.add_subplot(1, 1, 1)
    ax.set_title('Advektion-Dispersion Transport', fontsize=14)
    ax.set_xlabel ('Zeit', fontsize=14)
    ax.set_ylabel ('Konzentration', fontsize=14)
    #ax.set(xlabel='Zeit', ylabel='Konzentration', title='A-D transport', fontsize=14)
        
# PLOT HERE
    if plot_A == 1:
        ax.plot(time,conca, 'fuchsia', linewidth=2, label="(nur) Advektion - berechnet")
    if plot_AD == 1:
        ax.plot(time,conc, 'navy', linewidth=2, label="Advektion-Dispersion - berechnet")
    if plot_DATA == 1:
        ax.plot(t_obs, c_obs, 'ro', label="Messwerte")
    #ax.scatter(t_obs, c_obs, marker="x", c="red", zorder=10)
    plt.ylim(0, cp)
    plt.xlim(0,t1)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    if not plot_A !=1 and plot_AD != 1 and plot_DATA != 1:
        plt.legend(frameon=False, loc='upper right', fontsize=14)
    #ax.grid()

    plt.show()
    
#THE FOLLOWING ALLOWS TO DEFINE THE RANGE OF SLIDERS DEPENDING ON OTHER SLIDER SETTINGS (HERE DISP AS FUNCTION OF LENGTH)
interact(transport,
         #DEFINE THE RANGE SUCH THAT NOT TOO MANY COMPUTATIONS ARE NECESSARY WHEN MOVING THE SLIDER!
         l  = FloatText(value=15,min=1, max=100,step=1,description='Länge', readout_format='.2f' ),
         t1 = widgets.IntText(value=1800, min = 60, max = 86400, step = 60, description = 'Zeit max'),
         c0 = widgets.FloatText(value=0.1,min=0.01, max=5,step=0.01,description='Zugabe Konz.', readout_format='.2f' ),
         m  = widgets.FloatText(value=10,min=0, max=1000,step=1,description='Zugabe Masse', readout_format='.2f' ),
         Q  = widgets.FloatText(value=0.2, min=0.00001, max=0.001, step=0.0001, description='Durchfluss Q',readout_format='.5f'),
         n  = widgets.FloatSlider(value=0.2,min=0.02, max=0.6, step=0.001, description='n',readout_format='.3f'),       
         a  = FloatSlider(value=0.01,min=0.001, max=10,step=0.002,description='alpha', readout_format='.3f' ),
         plot_A    = widgets.Checkbox(value=True, description='Plot nur Advektion ',disabled=False),
         plot_AD   = widgets.Checkbox(value=False, description='Plot Advektion-Dispersion ',disabled=False),
         plot_DATA = widgets.Checkbox(value=False, description='Plot gemessene Daten ',disabled=False),
        )

interactive(children=(FloatText(value=15.0, description='Länge', step=1.0), IntText(value=1800, description='Z…

<function __main__.transport(l, t1, c0, m, Q, n, a, plot_A, plot_AD, plot_DATA)>

1. Im ersten Schritt können Sie mit der analytischen Lösung für Advektiven Transport experimentieren. Die Lösung verändert sich, wenn Sie die Porosität modifizieren. Ebenso können Sie über Konzentration und Masse das Eingangssignal modifizieren (Beachten Sie aber, dass die gemessenen Werte bei einer Konzentration von $c = 0,1 g m^{-3}$ sowie einer Zugabemenge von $m = 10 g$ gemessen worden sind).

2. Im zweiten Schritt können Sie sich die Funktion mit zusätzlicher Dispersion berechnen und plotten. Untersuchen Sie, wie sich die Dispersion auf die Konzentration auswirkt.

3. Letztendlich können Sie die gemessenen Werte einzeichnen und versuchen, den berechneten Verlauf der Konzentration optimal an die Messwerte anzupassen. Die dabei ermittelten Parameter repräsentieren das Transportverhalten des untersuchten Sediments.

In [156]:
Frage1 = "Was erwarten Sie, wenn Sie den Porenraum / die Porosität reduzieren?"
A1_A = "Die Konzentration bricht eher durch"
A1_B = "Die Konzentration bricht später durch"
A1_C = "Die Konzentration steigt an"
A1_D = "Die Konzentration sinkt ab"

Frage2 = "Sie berechnen den Transport mit Advektion und Dispersion. Was erwarten Sie? Im Verglich zu reiner Advektion ..."
A2_A = "nimmt die transportierte Stoffmenge (Fracht) ab"
A2_B = "gelöste Stoffe erreichen später Durchbruch"
A2_C = "gelöste Stoffe erreichen früher Durchbruch"
A2_D = "nimmt die transportierte Stoffmenge (Fracht) zu"

Frage3 = "Die Dispersivität ist eine skalenabhängige Größe. Laut Faustregel beträgt die Dispersivität im Vergleich zur Transportstrecke (hier Länge)..."
A3_A = "5%"
A3_B = "10%"
A3_C = "30%"
A3_D = "50%"

Frage4 = "Berechnung Adv.-Disp.: Sie geben die gelösten Stoffe länger zu (in Berechnung Masse zugegebener Stoff erhöhen). Was erwarten Sie?"
A4_A = "Durchbruch infolge Dispersion ändert sich."
A4_B = "Berechnete Konz. nähert sich Zugabekonz."
A4_C = "Konz. am Durchbruch steigt"
A4_D = "Konz. am Durchbruch steigt"

Q1 = create_multipleChoice_widget_PDF(Frage1,[A1_A,A1_B,A1_C,A1_D],A1_A)
Q2 = create_multipleChoice_widget_PDF(Frage2,[A2_A,A2_B,A2_C,A2_D],A2_C)
Q3 = create_multipleChoice_widget_PDF(Frage3,[A3_A,A3_B,A3_C,A3_D],A3_B)
Q4 = create_multipleChoice_widget_PDF(Frage4,[A4_A,A4_B,A4_C,A4_D],A4_B)

display(Q1)
display(Q2)
display(Q3)
display(Q4)

<hr>
letzte Änderung 2021 12 08 durch Thomas.Reimann@tu-dresden.de

&copy; 2021 | Die Autoren
<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img style="float: right" alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>